# Data1

### Read csv

In [2]:
import pandas as pd

data1 = pd.read_csv('data1/employees.csv', sep=';')
data1.columns
print(data1.shape)
data1.head()

(107, 11)


,employee_id,first_name,last_name,email,phone_number,hire_date,job_id,salary,commission_pct,manager_id,department_id
0,108,Nancy,Greenberg,NGREENBE,515.124.4569,1994-08-17 00:00:00,FI_MGR,12000.0,NaN,101.0,100
1,109,Daniel,Faviet,DFAVIET,515.124.4169,1994-08-16 00:00:00,FI_ACCOUNT,9000.0,NaN,108.0,100
2,110,John,Chen,JCHEN,515.124.4269,1997-09-28 00:00:00,FI_ACCOUNT,8200.0,NaN,108.0,100
3,111,Ismael,Sciarra,ISCIARRA,515.124.4369,1997-09-30 00:00:00,FI_ACCOUNT,7700.0,NaN,108.0,100
4,112,Jose Manuel,Urman,JMURMAN,515.124.4469,1998-03-07 00:00:00,FI_ACCOUNT,7800.0,NaN,108.0,100


### Find length of VARCHAR attributes

In [3]:
maxLenFirstName = 0
maxLenLastName = 0
maxLenPhoneNumber = 0
maxLenJobId = 0
for row in data1.values:
  maxLenFirstName=max(maxLenFirstName,len(row[1]))
  maxLenLastName=max(maxLenLastName,len(row[2]))
  maxLenPhoneNumber=max(maxLenPhoneNumber,len(row[4]))
  maxLenJobId=max(maxLenJobId,len(row[6]))
print(maxLenFirstName, maxLenLastName, maxLenPhoneNumber, maxLenJobId)

11 11 18 10


### Set of values for partition column 

In [4]:
data1['department_id'].drop_duplicates()

0     100
6      50
31     80
Name: department_id, dtype: int64

# Data2

In [6]:
data2 = pd.read_csv('data2/employees.csv', sep=',')
data2.columns
print(data2.shape)
data2.head()

(331603, 8)


,emp_no,birth_date,first_name,last_name,gender,hire_date,dept_no,from_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,d005,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,d007,1996-08-03
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,d004,1995-12-03
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,d004,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12,d003,1989-09-12


In [7]:
sorted(data2['dept_no'].drop_duplicates())

['d001', 'd002', 'd003', 'd004', 'd005', 'd006', 'd007', 'd008', 'd009']

In [9]:
salaryData = pd.read_csv('data2/salaries.csv')
salaryData.shape

(2844047, 4)

### Find partition vector for salary for each date interval

In [33]:
employees = pd.read_csv('data2/employees-with-salary.csv')
minValue = employees['hire_date'].min()
print(minValue)
maxValue = employees['hire_date'].max()
print(maxValue)
employees.head()

1985-01-01
2000-01-28


,emp_no,birth_date,first_name,last_name,gender,hire_date,dept_no,from_date,salary
0,47830,1954-04-21,Eben,Erni,M,1995-10-21,d004,1998-08-02,70792
1,48491,1961-08-20,Uwe,Azuma,F,1986-01-17,d007,1986-01-17,96929
2,49651,1958-11-06,Ohad,Badr,F,1990-12-21,d005,1999-03-26,74598
3,57211,1953-07-09,Uta,Undy,M,1985-03-01,d005,1985-03-01,73006
4,58036,1958-02-23,Henk,Gist,M,1994-03-07,d004,1998-02-04,46718


In [31]:
import numpy as np
import math

partVec = ['1988-01-01', '1994-01-01']
n = len(partVec) + 1
m = 2 # length of subpartition vector
subParts = np.zeros((n, m), dtype=int)
for i in range(n):
  if i == 0:
    salaries = employees[employees['hire_date'] < partVec[i]]['salary'].to_numpy()
    salaries.sort()
  elif i == n - 1:
    salaries = employees[employees['hire_date'] > partVec[i - 1]]['salary'].to_numpy()
    salaries.sort()
  else:
    salaries = employees[(employees['hire_date'] < partVec[i]) & (employees['hire_date'] > partVec[i - 1])]['salary'].to_numpy()
    salaries.sort()

  cnt = len(salaries)
  for j in range(m):
    subParts[i][j] = salaries[math.floor(cnt * (j + 1) / (m + 1)) - 1]

print(subParts)


[[66342 80853]
 [60286 74177]
 [52667 66268]]
